Вводная к проекту:
В ходе выполнения проекта мы разоберемся влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd #импортируем необходимые библиотеки

In [15]:
df = pd.read_csv(r'C:\Users\aglagoleva\Downloads\data1.csv') # загружаем датасет
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [16]:
df.info() #просматриваем информацию о данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [17]:
df.columns #проверяем названия колонок

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

### Шаг 2. заполнение пропусков

In [18]:
df.isna().sum() #смотрим наличие пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [19]:
share_days_employed_nan = 2174/21525
share_total_income_nan = 2174/21525
print('Доля пропущенных значений в столбце общий трудовой стаж в днях: {:.2%}'.format(share_days_employed_nan))
print('Доля пропущенных значений в столбце общий доход: {:.2%}'.format(share_total_income_nan))

Доля пропущенных значений в столбце общий трудовой стаж в днях: 10.10%
Доля пропущенных значений в столбце общий доход: 10.10%


In [20]:
median_days_employed = df['days_employed'].median()
median_days_employed

-1203.369528770489

In [21]:
median_total_income = df['total_income'].median()
median_total_income

145017.93753253992

In [22]:
df['days_employed'] = df['days_employed'].fillna(median_days_employed)
df['total_income'] = df['total_income'].fillna(median_total_income)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Предобработка данных:
Мы имеем датасет размером 21525 Х 12. 
1) Сразу бросается в глаза аномальные сведения - в столбце 'days_employed'(общий трудовой стаж в днях) стоят отрицательные значения
2) В двух колонках имеем пропуски значений. количество пропусков одинаковое - они связаны друг с другом. потому что зп напрямую зависит от кол-ва рабочих дней. Скорее всего объект не работал, поэтому пропуски в данных, или работал не официально
3) Мы заменили все пропуски на медианные значения столбцов 

Заполняю пропуски медианой, чтобы заменить аномальные значения. Поскольку медиана - это серединное значение набора чисел. Для заполнения берут медиану, потому что она нечувствительна к выбросам

In [24]:
df.isna().sum() #смотрим наличие пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [25]:
share_days_employed_nan = 2174/21525
share_total_income_nan = 2174/21525
print('Доля пропущенных значений в столбце общий трудовой стаж в днях: {:.2%}'.format(share_days_employed_nan))
print('Доля пропущенных значений в столбце общий доход: {:.2%}'.format(share_total_income_nan))

Доля пропущенных значений в столбце общий трудовой стаж в днях: 10.10%
Доля пропущенных значений в столбце общий доход: 10.10%


In [26]:
median_days_employed = df['days_employed'].median()
median_days_employed

-1203.369528770489

In [27]:
median_total_income = df['total_income'].median()
median_total_income

145017.93753253992

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [28]:
 df['days_employed'] = df['days_employed'].abs() #трансформируем отрицательные значения в положительные, взяв модуль числа

In [29]:
median = df['days_employed'].median() #определяем медиану
median

1808.0534339280623

In [30]:
df.loc[df['days_employed'] > median, 'days_employed'] = median #заменяем все значения, которые больше медианы на медиану 

Заменяю большИе значения медианными: человек не может работать больше дней чем жил, или все свои прожитые дни. оптимально будет заменить их медианой

In [31]:
df_gr = df.groupby('income_type')['days_employed'].count() #группируем количество рабочих дней по категории, не знаю зачем я это сделала, наверное, потому что просто могу, но этому предшествовала кака-то хорошая идея, которая ушла
df_gr

income_type
безработный            2
в декрете              1
госслужащий         1459
компаньон           5085
пенсионер           3856
предприниматель        2
сотрудник          11119
студент                1
Name: days_employed, dtype: int64

In [32]:
df['children'].value_counts() # поиск аномалий в колонке "дети" 

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [33]:
#мы увидели, выбросы в виде "-1" и "20", заменим это на 1 и на 2

In [34]:
df.loc[df['children'] == -1, 'children'] = 1
df.loc[df['children'] == 20, 'children'] = 2
df['children'].value_counts() #видим, что аномалии устранены

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [35]:
df['dob_years'].value_counts() #видим, что аномалии устранены

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

видим "нулевой возраст", уберем его. заменив на медианное значение

In [36]:
df['dob_years'].median()

42.0

In [37]:
df.loc[df['dob_years'] == 0, 'dob_years'] = 42
df['dob_years'].value_counts()

42    698
35    617
40    609
41    607
34    603
38    598
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Видим, что ноль изчез, можем двигаться дальше

### Шаг 2.3. Изменение типов данных.

In [38]:
df['total_income'] = df['total_income'].astype('int')

In [39]:
df.isna().sum() #смотрим наличие пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [40]:
share_days_employed_nan = 2174/21525
share_total_income_nan = 2174/21525
print('Доля пропущенных значений в столбце общий трудовой стаж в днях: {:.2%}'.format(share_days_employed_nan))
print('Доля пропущенных значений в столбце общий доход: {:.2%}'.format(share_total_income_nan))

Доля пропущенных значений в столбце общий трудовой стаж в днях: 10.10%
Доля пропущенных значений в столбце общий доход: 10.10%


In [42]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,1808.053434,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,1808.053434,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,1808.053434,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,1808.053434,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,1808.053434,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [43]:
median_days_employed = df['days_employed'].median()
median_days_employed

1808.0534339280623

In [44]:
median_total_income = df['total_income'].median()
median_total_income

145017.0

### Шаг 2.4. Удаление дубликатов.

In [45]:
df.duplicated().sum() #выявляем количество явных дубликатов

55

In [46]:
print(df['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


In [47]:
df['education'] = df['education'].str.lower() #приводим столбец "образование" к нижниму регистру

In [48]:
df.duplicated().sum() #проверяем наличие дубликатов снова

72

In [51]:
df = df.drop_duplicates().reset_index(drop = True) #удаляем дубликаты

In [52]:
df.duplicated().sum() #проверяем снова кол-во дубликатов

0

In [53]:
df['education'].value_counts()

среднее                15171
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

были обнаружены явные дубликаты, а так же приведя данные в колонке "образование" к единому регистру выявили неявные дубликаты, и удалены

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [54]:
family_status_df = df[['family_status', 'family_status_id']]
family_status_df = family_status_df.drop_duplicates().reset_index(drop = True)
family_status_df

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [55]:
education_df = df[['education', 'education_id']]
education_df = education_df.drop_duplicates().reset_index(drop = True)
education_df

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [41]:
#удаляем
df = df.drop(['family_status', 'education'], axis=1)

### Шаг 2.6. Категоризация дохода.

In [57]:
#назначаем категории исходя из диапазонов доходов:
def income_category(row):
    income = row['total_income']
    if income <= 30000:
        return 'E'
    elif 30001<= income<=50000:
        return 'D'
    elif 50001<= income<= 200000:
        return 'C'
    elif 200001<= income <= 1000000:
        return 'B'
    elif income >= 1000001:
        return 'A'

In [58]:
df['total_income_category'] = df.apply(income_category, axis = 1)

In [59]:
print(df['total_income_category'].value_counts())

C    16015
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64


### Шаг 2.7. Категоризация целей кредита.

In [60]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [61]:
# назначаем категории целей на кредит
def purpose_cat(row):
    purpose = row['purpose']
    if ('жил' or 'нед') in purpose:
        return 'операции с недвижимостью'
    elif 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'свад' in purpose:
        return 'проведение свадьбы'
    elif 'образ' in purpose:
        return 'получение образования'

In [62]:
df['purpose_category'] = df.apply(purpose_cat, axis = 1)

In [63]:
print(df['purpose_category'].value_counts())

операции с недвижимостью    5690
операции с автомобилем      4306
получение образования       4013
проведение свадьбы          2323
Name: purpose_category, dtype: int64


In [64]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,1808.053434,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,1808.053434,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,1808.053434,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,1808.053434,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,1808.053434,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


Нашли аномалии, заменили их на подходящие значения
Нашли явные и неявны дубликаты - удалили
сформировали дополнительные датафрейм словари, сделали декомпозицию исходного датафрейма

### Ответы на вопросы.

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [70]:
data_pivot_children = df.pivot_table(index=['children'], values=["debt"], aggfunc=['sum', 'count', 'mean']).reset_index(drop = True)

data_pivot_children = data_pivot_children.sort_values(by=('mean', 'debt')).round(2)
data_pivot_children

,sum,count,mean
,debt,debt,debt
5,0,9,0.00
0,1063,14090,0.08
3,27,330,0.08
1,445,4855,0.09
2,202,2128,0.09
4,4,41,0.10


из таблицы видим, что в процентном соотношении вероятность задолжности чуть ниже, чем у всех у кого дети есть, но у семей с тремя детьми тоже ниже вероятность. (извините, последний столбец было бы хорошо умножить на сто, чтобы было более красивее, но я не знаю как это сейчас сделать, поэтому пока так, извините)


Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [69]:
data_pivot_family_status = df.pivot_table(index=['family_status_id'], values=["debt"], aggfunc=['sum', 'count', 'mean']).reset_index(drop = True)

data_pivot_family_status = data_pivot_family_status.sort_values(by=('mean', 'debt')).round(2)
data_pivot_family_status


,sum,count,mean
,debt,debt,debt
2,63,959,0.07
3,85,1195,0.07
0,931,12339,0.08
1,388,4150,0.09
4,274,2810,0.10


In [71]:
family_status_df

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


видим, что люди которые состоят в отношениях или состоли, более обязательные, чем люди, которые не состоят в каком либо браке

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [74]:
data_pivot_income_cat = df.pivot_table(index=['total_income_category'], values=["debt"], aggfunc=['sum', 'count', 'mean']).reset_index(drop = True)

data_pivot_income_cat = data_pivot_income_cat.sort_values(by=('mean', 'debt'))
data_pivot_income_cat

,sum,count,mean
,debt,debt,debt
3,21,350,0.060000
1,356,5041,0.070621
0,2,25,0.080000
2,1360,16015,0.084920
4,2,22,0.090909


люди с очень маленьким доходом и с очень большим доходом не охотно берут кредиты, соответсвенно и выплачивают их быстрее

Как разные цели кредита влияют на его возврат в срок?

In [75]:
data_pivot_purpose_category = df.pivot_table(index=['purpose_category'], values=["debt"], aggfunc=['sum', 'count', 'mean'])

data_pivot_purpose_category = data_pivot_purpose_category.sort_values(by=('mean', 'debt'))
data_pivot_purpose_category

,sum,count,mean
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,397,5690,0.069772
проведение свадьбы,186,2323,0.080069
получение образования,370,4013,0.092200
операции с автомобилем,403,4306,0.093590


# ОТВЕТЫ НА ВОПОРСЫ

Есть ли зависимость между количеством детей и возвратом кредита в срок?
Да, чем меньше детей, тем быстрее они возвращают кредиты

Есть ли зависимость между семейным положением и возвратом кредита в срок?
да, те кто не обременены браком возвращают деньги дольше, люди состоящие в отношениях или вышедшие их них (развод или потеря партнера)они более ответсвенные

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
да, доля задолжностей у самых бедных и самых богатых - минимальныя, среди тех кто по середени - чаще появляются проблемы с кредитами 


Как разные цели кредита влияют на его возврат в срок?
Как оказалось за свадьбы и квартиры люди выплачивают кредиты быстрее, без просрочек


## Общий вывод:

люди в браке со среднем достатком чаще имеют задолженность по возврату кредитов